# 1. Import Data

In [ ]:
from obspy import read
import numpy
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import scipy.signal as signal

from functions_MA import roof_edges3D, set_axes_equal, roof_edges2D, H_array2D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from numpy import sin, cos, pi
import matplotlib.patches as patches

%matplotlib nbagg

In [ ]:
root2data = 'PrimeTower_data'
acc_orig = read('%s/2021.10.1.9E.ZPT.R1.HGE' % root2data)
acc_orig += read('%s/2021.10.1.9E.ZPT.R1.HGN' % root2data)
acc_orig += read('%s/2021.10.1.9E.ZPT.R1.HGZ' % root2data)

rot_rate_orig = read('%s/2021.10.1.9E.ZPT.R1.HJE' % root2data)
rot_rate_orig += read('%s/2021.10.1.9E.ZPT.R1.HJN' % root2data)
rot_rate_orig += read('%s/2021.10.1.9E.ZPT.R1.HJZ' % root2data)

# 2. Pre-Processing


In [ ]:
rot_rate = rot_rate_orig.copy()
acc = acc_orig.copy()

freq = 0.08
df = 50

# filter out low frequencies
rot_rate.detrend('simple').filter('highpass', freq = freq, zerophase = True)
acc.detrend('simple').filter('highpass', freq = freq, zerophase = True)

# filter out high frequencies (25Hz) and resample to 50 Hz
rot_rate.filter('lowpass', freq = 25, zerophase = True).resample(sampling_rate=df)
acc.filter('lowpass', freq = 25, zerophase = True).resample(sampling_rate=df)

# integrate to nrad and m
angle = rot_rate.filter('highpass', freq = freq, zerophase = True).integrate().filter('highpass', freq = freq, zerophase = True)
disp = acc.filter('highpass', freq = freq, zerophase = True).integrate().integrate().filter('highpass', freq = freq, zerophase = True)

print(disp[0].stats, angle[0].stats)

# 3. Plotting Timeseries

In [ ]:
data_t = disp.copy() # acc
unit_t = 'm'
data_r = angle.copy() # rot_rate
unit_r = 'nrad' # the rotation data is already in nrad

fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(14,8), sharex=True)
plt.subplots_adjust(hspace=0, wspace=0.2)
# plot rotations
for ax, ch in zip(axs[:,0], ['E','N','Z']):
    ax.plot(data_r.select(channel='HJ%s' %ch)[0].times('matplotlib'),data_r.select(channel='HJ%s' %ch)[0].data, color='red')
    ax.set_ylabel(ch+' ['+ unit_r + '] ')
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
# plot translations
for ax, ch in zip(axs[:,1], ['E','N','Z']):
    ax.plot(data_t.select(channel='HG%s' %ch)[0].times('matplotlib'),data_t.select(channel='HG%s' %ch)[0].data, color='black')
    ax.set_ylabel(ch+' ['+ unit_t + '] ')
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
    
axs[2,0].set_xlabel('Time')
axs[2,1].set_xlabel('Time')

# Adjust format as needed
fig.autofmt_xdate() 

plt.show()

# 4. Plot Frequency domain

In [ ]:

data_t = disp.copy() 
unit_t = 'm^2/Hz'
data_r = angle.copy() 
unit_r = 'nrad^2/Hz'
dt = data_t[0].stats.delta

# define the number per segment for the welch power spectral density calculation
scale_length = 5

fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(14,8), sharex=True)
plt.subplots_adjust(hspace=0, wspace=0.2)
# plot rotations
length = int(len(data_r.select(channel='HJE')[0].data)/scale_length)
for ax, ch in zip(axs[:,0], ['E','N','Z']):
    f_r, P_r = signal.welch(data_r.select(channel='HJ%s' % ch)[0].data, 1 / dt, nperseg=length)
    ax.loglog(f_r,P_r, color='red')
    ax.set_ylabel(ch+' ['+ unit_r + '] ')
    ax.set_ylim(top=2e7, bottom=1e-1)
    ax.grid(which='major', axis='both')
# plot translations
length = int(len(data_t.select(channel='HGE')[0].data)/scale_length)
for ax, ch in zip(axs[:,1], ['E','N','Z']):
    f_t, P_t = signal.welch(data_t.select(channel='HG%s' % ch)[0].data, 1 / dt, nperseg=length)
    ax.loglog(f_t,P_t, color='black')
    ax.set_ylabel(ch+' ['+ unit_t + '] ')
    ax.set_ylim(top=2e-7, bottom=1e-18)
    ax.grid(which='major', axis='both')
    
axs[2,0].set_xlabel('Frequency [Hz]')
axs[2,1].set_xlabel('Frequency [Hz]')

axs[0,0].set_xlim(left=0.1, right=10)

  # Adjust format as needed
fig.autofmt_xdate() 

plt.show()
#fig.savefig('PSD.png', dpi=300, bbox_inches='tight')


# 5. Prepare for Peak Picking
In this section we prepare for the peak picking. 
1. identify the approximate modal frequencies
2. define a band around these frequencies
3. identify the main motion direction
4. check that the frequencies and the bands are done correctly

For number 3 it is important to pick a direction that is clearly above the sensor noise, so that we don't end up picking noise.

In [ ]:
data_t = disp.copy() 
unit_t = 'm^2/Hz'
data_r = angle.copy() 
unit_r = 'nrad^2/Hz'
dt = data_t[0].stats.delta

# define the number per segment for the welch power spectral density calculation
scale_length = 5
#scale_length = 

 # 1. identify the approximate modal frequencies
#f_oI = [,,,,,]

# 2. define a band around these frequencies
#f_band = [[, ], [,], [,], [,], [,], [,]]

# 3. identify the main motion direction
#main_ch_allf = [,,,,,]


ch = ['HGE','HGN','HGZ']
chr = ['HJE','HJN','HJZ']

# 4. check that the frequencies and the bands are done correctly
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(14,8), sharex=True)
plt.subplots_adjust(hspace=0, wspace=0.2)
# plot rotations
length = int(len(data_r.select(channel='HJE')[0].data)/scale_length)
for ax, ch in zip(axs[:,0], ['E','N','Z']):
    f_r, P_r = signal.welch(data_r.select(channel='HJ%s' % ch)[0].data, 1 / dt, nperseg=length)
    ax.loglog(f_r,P_r, color='red')
    ax.set_ylabel(ch+' ['+ unit_r + '] ')
    ax.set_ylim(top=2e7, bottom=1e-1)
    ax.grid(which='major', axis='both')
    for f in f_oI:
        ax.axvline(f, color='grey')
    for fmin, fmax in f_band:
        ax.axvline(fmin, color='lightblue')
        ax.axvline(fmax, color='darkblue')
# plot translations
length = int(len(data_t.select(channel='HGE')[0].data)/scale_length)
for ax, ch in zip(axs[:,1], ['E','N','Z']):
    f_t, P_t = signal.welch(data_t.select(channel='HG%s' % ch)[0].data, 1 / dt, nperseg=length)
    ax.loglog(f_t,P_t, color='black')
    ax.set_ylabel(ch+' ['+ unit_t + '] ')
    ax.set_ylim(top=2e-7, bottom=1e-18)
    ax.grid(which='major', axis='both')
    for f in f_oI:
        ax.axvline(f, color='grey')
    for fmin, fmax in f_band:
        ax.axvline(fmin, color='lightblue')
        ax.axvline(fmax, color='darkblue')
    
axs[2,0].set_xlabel('Frequency [Hz]')
axs[2,1].set_xlabel('Frequency [Hz]')

axs[0,0].set_xlim(left=0.1, right=10)

# Adjust format as needed
fig.autofmt_xdate() 

plt.show()

# 6. Frequency Picking on each axis

The objective here is to estimate the frequency of each of the first six modes. We use the maximum PSD amplitude of the peak of the mode to define the frequency thereof.

1. calculate power spectral density using signal.welch() to get amplitudes "P' and frequencies "f".
2. get indexes of the frequencies defined in "f_band".
3. get the index of the maximum amplitude of power spectral density within that frequency band.
4. use that index to get frequency at that point.
5. repeat 3 times for each channel east, north, Z
6. repeat 6 times for each of the six modes

## 6.1 Calculations

In [ ]:
data_t = disp.copy() 
unit_t = 'm^2/Hz'
data_r = angle.copy() 
unit_r = 'nrad^2/Hz'
dt = data_t[0].stats.delta
# define the number per segment for the welch power spectral density calculation
scale_length = 5

save_matrix_each = []

ch = ['HGE','HGN','HGZ']
chr = ['HJE','HJN','HJZ']
axis = ['E','N','Z']

for n in range(6): # looping through all six modes (0,5)
    for i in range(3): # looping through all three axes (east, north, up)
        ch_t = ch[i]
        ch_r = chr[i]
        freqmin, freqmax = f_band[n][0], f_band[n][1]   # Frequency band limits
        
        #######################################################################################
        # Translation:
        # 1. calculate power spectral density using signal.welch() to get amplitudes "P' and frequencies "f".
        length = int(len(data_t.select(channel=ch_t)[0].data)/scale_length)
        f_t, P_t = signal.welch(data_t.select(channel=ch_t)[0].data, 1 / dt, nperseg=length)
        # 2. get indexes of the frequencies defined in "f_band".
        index_fmin = numpy.abs(f_t - freqmin).argmin()
        index_fmax = numpy.abs(f_t - freqmax).argmin()
        
        # Identify peak amplitude within range
        # 3. peak amplitude and relative index
        single_peak_ampl_t = numpy.max(P_t[index_fmin:index_fmax+1])
        index_peak_ampl_t = numpy.where(P_t[index_fmin:index_fmax+1] == single_peak_ampl_t)[0][0]
        # peak index:
        new_middle = index_fmin + index_peak_ampl_t
    
        # peak amplitude of top 3 picks
        peak_ampl_t = numpy.mean(P_t[new_middle - 1:new_middle + 2])
        # 4. frequency corresponding to peak amplitude
        peak_f_t = f_t[new_middle]
        
        #######################################################################################
        # Rotation:
        # 1. calculate power spectral density using signal.welch() to get amplitudes "P' and frequencies "f".
        length = int(len(data_r.select(channel=ch_r)[0].data)/scale_length)
        f_r, P_r = signal.welch(data_r.select(channel=ch_r)[0].data, 1 / dt, nperseg=length)
        # 2. get indexes of the frequencies defined in "f_band".
        index_fmin = numpy.abs(f_r - freqmin).argmin()
        index_fmax = numpy.abs(f_r - freqmax).argmin()
    
        # Identify peak amplitude within range
        # 3. peak amplitude and relative index
        single_peak_ampl_r = numpy.max(P_r[index_fmin:index_fmax+1])
        index_peak_ampl_r = numpy.where(P_r[index_fmin:index_fmax+1] == single_peak_ampl_r)[0][0]
        # peak index:
        new_middle = index_fmin + index_peak_ampl_r
    
        # peak amplitude of top 3 picks
        peak_ampl_r = numpy.mean(P_r[new_middle - 1:new_middle + 2])
        # 4. frequency corresponding to peak amplitude
        peak_f_r = f_r[new_middle]

        # Format timestamp for output
        start_str = str(data_r[0].stats.starttime)
        newfilename = '' + start_str[0:4] + start_str[5:7] + start_str[8:10] + start_str[11:13] + start_str[14:16] + start_str[17:19] + ''
        # date&time, [x,y,z],        mode,           mean ampl trans, mean ampl rot, f trans, f rot
        #  float   , [E,N,Z],  [0, 1, 2, 3, 4, 5],        float     ,   float      ,   float, float
        #  0,           1,            2,                  3,               4,           5,       6
        save_line = [float(newfilename),axis[i],n,numpy.sqrt(peak_ampl_t*peak_f_t),numpy.sqrt(peak_ampl_r*peak_f_r), peak_f_t, peak_f_r]
        #save_line = [float(newfilename),,,,, , ]
        save_matrix_each.append(save_line)
ma_picks_individual = numpy.asarray(save_matrix_each)
print(ma_picks_individual)

## 6.2 Plotting the frequencies

In [ ]:
# 6 modes, 6 frequency estimates

fig, axs = plt.subplots(1,6)
plt.subplots_adjust(wspace=0.1)

# Define translation and rotation channels for each mode
channel_E_t = ['HGE'] * 6
channel_N_t = ['HGN'] * 6
channel_Z_t = ['HGZ'] * 6
channel_E_r = ['HJE'] * 6
channel_N_r = ['HJN'] * 6
channel_Z_r = ['HJZ'] * 6

# Extract frequency values from ma_picks_individual dataset
E_t = ma_picks_individual[ma_picks_individual[:,1] == 'E'][:,5].astype(float)
N_t = ma_picks_individual[ma_picks_individual[:,1] == 'N'][:,5].astype(float)
Z_t = ma_picks_individual[ma_picks_individual[:,1] == 'Z'][:,5].astype(float)

E_r = ma_picks_individual[ma_picks_individual[:,1] == 'E'][:,6].astype(float)
N_r = ma_picks_individual[ma_picks_individual[:,1] == 'N'][:,6].astype(float)
Z_r = ma_picks_individual[ma_picks_individual[:,1] == 'Z'][:,6].astype(float)

# Scatter plot of frequency estimates
for freq, channel in zip([E_t, N_t, Z_t, E_r, N_r, Z_r], [channel_E_t, channel_N_t, channel_Z_t, channel_E_r, channel_N_r, channel_Z_r]):
    for f, ch, ax in zip(freq, channel, axs):
        ax.scatter(f, ch)

# Configure x-axis limits and formatting
for ax, n in zip(axs,range(6)):
    ax.grid(axis='both')
    ax.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    labelrotation=45,
    grid_linewidth=0.4,
    grid_color='lightgrey')
    ax.set_xlim(left = f_band[n][0]-0.01,right=f_band[n][1]+0.01)
# Configure y-axis formatting
for ax in axs[1:]:
    ax.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    left=False,
    labelleft=False,
    grid_linewidth=0.4,
    grid_color='lightgrey')

plt.show()


# 7. Amplitude Peak Picking using only main frequency
The objective here is to get the amplitude of each of the 3 components of both translation and rotation to get 6C modeshapes. For this we only determine the frequency on the main direction of motion and use that frequency on all other channels to get the amplitudes.
1. extract main direction of motion for the mode
2. for that direction of motion calculate power spectral density using signal.welch() to get amplitudes "P' and frequencies "f".
2. get indexes of the frequencies defined in "f_band".
3. get the index of the maximum amplitude of power spectral density within that frequency band.
4. use that index to get the amplitude of each of the six components 
5. use that index to get frequency of each of the six components 
6. repeat 3 times for each component of translation and rotation
7. repeat 6 times for each of the six modes


In [ ]:
data_t = disp.copy() 
unit_t = 'm^2/Hz'
data_r = angle.copy() 
unit_r = 'nrad^2/Hz'
dt = data_t[0].stats.delta
# define the number per segment for the welch power spectral density calculation
scale_length = 5

save_matrix = []

for n in range(6): # looping through all six modes (0,5)
    main_ch = main_ch_allf[n] # 1. Main direction of motion
    freqmin, freqmax = f_band[n][0], f_band[n][1]   # Frequency band limits
    
    if 'G' in main_ch: # Translation component
        # 2. calculate power spectral density using signal.welch() to get amplitudes "P' and frequencies "f".
        length = int(len(data_t.select(channel=main_ch)[0].data)/scale_length)
        f_t, P_t = signal.welch(data_t.select(channel=main_ch)[0].data, 1 / dt, nperseg=length)
        # 3. index around fundamental freq
        index_fmin = numpy.abs(f_t - freqmin).argmin()
        index_fmax = numpy.abs(f_t - freqmax).argmin()+1

        # Identify peak amplitude within range
        # 4. peak amplitude and relative index
        single_peak_ampl_t = numpy.max(P_t[index_fmin:index_fmax])
        index_peak_ampl_t = numpy.where(P_t[index_fmin:index_fmax] == single_peak_ampl_t)[0][0]
        # peak index:
        new_middle = index_fmin + index_peak_ampl_t

    elif 'J' in main_ch: # Rotation component
        # 2. calculate power spectral density using signal.welch() to get amplitudes "P' and frequencies "f".
        length = int(len(data_r.select(channel=main_ch)[0].data)/scale_length)
        f_r, P_r = signal.welch(data_r.select(channel=main_ch)[0].data, 1 / dt, nperseg=length)
        # 3. index around fundamental freq
        index_fmin = numpy.abs(f_r - freqmin).argmin()
        index_fmax = numpy.abs(f_r - freqmax).argmin()+1

        # Find frequency index range
        # 4. peak amplitude and relative index
        single_peak_ampl_r = numpy.max(P_r[index_fmin:index_fmax])
        index_peak_ampl_r = numpy.where(P_r[index_fmin:index_fmax] == single_peak_ampl_r)[0][0]
        # peak index:
        new_middle = index_fmin + index_peak_ampl_r
    
    # Extract peak amplitudes for all translation and rotation components
    axis = ['E','N','Z']
    ch = ['HGE','HGN','HGZ']
    chr = ['HJE','HJN','HJZ']
    for i in range(3):
        # transform to frequency domain
        length = int(len(data_r.select(channel=chr[i])[0].data)/scale_length)
        f_r, P_r = signal.welch(data_r.select(channel=chr[i])[0].data, 1 / dt, nperseg=length)
        f_t, P_t = signal.welch(data_t.select(channel=ch[i])[0].data, 1 / dt, nperseg=length)

        # Translation:
        # 4. peak amplitude of top 3 picks
        peak_ampl_t = numpy.mean(P_t[new_middle - 1:new_middle + 2])
        # 5. frequency corresponding to peak amplitude
        peak_f_t = f_t[new_middle]

        # Rotation:
        # 4. peak amplitude of top 3 picks
        peak_ampl_r = numpy.mean(P_r[new_middle - 1:new_middle + 2])
        # 5. frequency corresponding to peak amplitude
        peak_f_r = f_r[new_middle]
        
        # Format timestamp for output
        start_str = str(data_r[0].stats.starttime)
        newfilename = '' + start_str[0:4] + start_str[5:7] + start_str[8:10] + start_str[11:13] + start_str[14:16] + start_str[17:19] + ''
        
        # Save frequency and amplitude estimations
        # date&time, [x,y,z],        mode,           mean ampl trans, mean ampl rot, f trans, f rot
        #            [E,N,Z],  [0, 1, 2, 3, 4, 5],                  ,              ,        ,    
        #  0,           1,            2,                  3,               4,           5,       6
        save_line = [float(newfilename),axis[i],n,numpy.sqrt(peak_ampl_t*peak_f_t),numpy.sqrt(peak_ampl_r*peak_f_r), peak_f_t, peak_f_r]
        #save_line = [float(newfilename),,,,, , ]
        save_matrix.append(save_line)
ma_picks = numpy.asarray(save_matrix)
print(ma_picks)

# 8. Plotting the Modeshapes and Frequencies.

## 8.1 Sign of modeshape


In [ ]:
data_t = disp.copy()  # acc
unit_t = 'm'
data_r = angle.copy()  # rot_rate
unit_r = 'nrad'

# choose n according to which mode you want to plot.
n=0 # 0-5

fmin, fmax = f_band[n][0], f_band[n][1]

fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(14,8), sharex=True)
plt.suptitle('mode %s Hz' % f_oI[n])
plt.subplots_adjust(hspace=0, wspace=0.2)
data_r_f = data_r.copy().filter('bandpass', freqmin = fmin, freqmax = fmax, zerophase=True)
data_t_f = data_t.copy().filter('bandpass', freqmin = fmin, freqmax = fmax, zerophase=True)
# plot rotations
for ax, ch in zip(axs[:], ['E','N','Z']):
    ax.plot(data_r_f.select(channel='HJ%s' %ch)[0].times('matplotlib'),data_r_f.select(channel='HJ%s' %ch)[0].data, color='red')
    ax.set_ylabel(ch+' ['+ unit_r + '] ')
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
# plot translations
for ax, ch in zip(axs[:], ['E','N','Z']):
    ax2 = ax.twinx()
    ax2.plot(data_t_f.select(channel='HG%s' %ch)[0].times('matplotlib'),data_t_f.select(channel='HG%s' %ch)[0].data, color='black')
    ax2.set_ylabel(ch+' ['+ unit_t + '] ')
    ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
    
axs[2].set_xlabel('Time')

# Adjust format as needed
fig.autofmt_xdate() 

#plt.show()

In [ ]:
# sign for modeshape
sign_t = [[-1,1,-1],[1,1,1],  [1,1,-1], [1,-1,1],[-1,-1,-1],[1,-1,1]]
#sign_t = [[,,], [,,], [,,], [,,], [,,], [,,]]
sign_r = [[-1,-1,1],[-1,1,-1],[1,-1,-1],[1,1,1], [-1,-1,1], [-1,1,1]]
#sign_r = [[,,], [,,], [,,], [,,], [,,], [,,]]

## 8.2 Plot Modeshape

In [ ]:


fig = plt.figure(figsize=(10, 5))

mult = 0.01
# pick either the first three modes 'pm' or the second three modes 'hom1'
#mode =

if mode == 'pm':
    mode_index = [0, 1, 2]
    scale_t = 500000
    scale_r = 0.001
    z_H_scale = 1
# or
elif mode == 'hom1':
    mode_index = [3, 4, 5]
    scale_t = 6000000
    scale_r = 0.003
    z_H_scale = 1

for m in [0,1,2]:
    # ma_picks
    # date&time, [x,y,z],        mode,           mean ampl trans, mean ampl rot, f trans, f rot
    #            [E,N,Z],  [0, 1, 2, 3, 4, 5],                  ,              ,        ,    
    #  0,           1,            2,                  3,               4,           5,       6
    single_MA_pick = ma_picks[ma_picks[:,2] == '%s' %mode_index[m],:] # get single mode picks
    indiv_freq_picks = ma_picks_individual[ma_picks_individual[:,2] == '%s' %mode_index[m],:] # get single mode picks
    
    # different directions
    E_pick = indiv_freq_picks[indiv_freq_picks[:,1] == 'E'][0]
    N_pick = indiv_freq_picks[indiv_freq_picks[:,1] == 'N'][0]
    Z_pick = indiv_freq_picks[indiv_freq_picks[:,1] == 'Z'][0]
    
    E_t = float(single_MA_pick[single_MA_pick[:,1] == 'E'][0][3]) * scale_t * sign_t[mode_index[m]][0]
    N_t = float(single_MA_pick[single_MA_pick[:,1] == 'N'][0][3]) * scale_t * sign_t[mode_index[m]][1]
    Z_t = float(single_MA_pick[single_MA_pick[:,1] == 'Z'][0][3]) * scale_t * sign_t[mode_index[m]][2]
    
    E_r = float(single_MA_pick[single_MA_pick[:,1] == 'E'][0][4]) * scale_r * sign_r[mode_index[m]][0]
    N_r = float(single_MA_pick[single_MA_pick[:,1] == 'N'][0][4]) * scale_r * sign_r[mode_index[m]][1]
    Z_r = float(single_MA_pick[single_MA_pick[:,1] == 'Z'][0][4]) * scale_r * sign_r[mode_index[m]][2]
    
    # load footprint of tower and plot roof and groundfloor
    edges_gfloor, edges_roof, ER, E23, E6, E0, BE0, ELon, ELat, Lon1, Lat1 = roof_edges3D(mult=mult)
    # make subplot figure and plot edges
    ax = fig.add_subplot(2, 3, m + 1, projection='3d')
    ax.add_collection3d(Poly3DCollection(edges_gfloor, edgecolor='k', facecolor=[1, 0, 0, 0]))
    ax.add_collection3d(Poly3DCollection(edges_roof, edgecolor='k', facecolor=[1, 0, 0, 0]))
    
    # define limits of plot, uses scaling defined above
    plt.xlim([-40 * mult, 40 * mult])
    plt.ylim([-40 * mult, 40 * mult])
    ax.set_zlim([65 * mult, 130 * mult])
    ax.set_title('%s, E_t = %s, E_r = %s Hz' % (f_oI[mode_index[m]], round(float(E_pick[5]), 3), round(float(E_pick[6]), 3)))
    

    # plot location of blueseis in black
    ax.plot(0, 0, 124.37 * mult, color='k', marker='o', zorder=19)
    #locations = H_array2D(mult=mult)

    ########################################################################################
    # plotting 2D, view from above.
    ax1 = fig.add_subplot(2, 3, m + 1 + 3)
    ax1.set_xlabel('East')
    ax1.set_ylabel('North')
    
    
    # load rotation modeshape angles and rotate the roof edges to make rotation mode shape
    for loop_scale,edgecolor, facecolor in zip([-1, -0.5, 0.5, 1], ['gainsboro', 'silver', 'gray', 'k'],
                                         ['plum', 'violet', 'mediumorchid', 'darkviolet']):
        
        # *-1 because the rotation of the marker is defined in different coordinate system..
        phi, theta, psi = E_r * loop_scale * -1, \
                          N_r * loop_scale * -1, \
                          Z_r * loop_scale * -1
        # Rotate the marker$
        rotater = numpy.array([[cos(theta) * cos(psi), cos(theta) * sin(psi), -sin(theta)],
                               [sin(phi) * sin(theta) * cos(psi) - cos(phi) * sin(psi),
                                sin(phi) * sin(theta) * sin(psi) + cos(phi) * cos(psi), cos(theta) * sin(phi)],
                               [cos(phi) * sin(theta) * cos(psi) + sin(phi) * sin(psi),
                                cos(phi) * sin(theta) * sin(psi) - sin(phi) * cos(psi), cos(theta) * cos(phi)]])
        Lat = N_t * loop_scale
        Lon = E_t * loop_scale
        
        edges_gfloor, edges, ER, E23, E6, E0, BE0, ELon, ELat, Lon1, Lat1 = roof_edges3D(mult=mult,
                                                                Lat=Lat,Lon=Lon, up = Z_t*loop_scale)
        edges_arr = numpy.asarray(edges_gfloor)
        edges_rot = edges_arr.copy()
        edges_rot2d = [[]]
        for i in range(8):
            array = rotater.dot(edges_arr[0][i])
            edges_rot[0][i] = [array[0], array[1], array[2] + 124.37 * mult]
            edges_rot2d[0].append([array[0], array[1]])
    
        # plot the 3D modeshapes
        ax.add_collection3d(Poly3DCollection(edges_rot, edgecolor=edgecolor, facecolor=facecolor))
        ax.set_box_aspect([2, 2, 1])
        plt.tight_layout()
        set_axes_equal(ax)
    
        # plotting 2D, view from above.
        patch1 = roof_edges2D(mult=mult)
        patch2 = numpy.asarray(edges_rot2d[0])
        poly1 = patches.Polygon(patch1, edgecolor='k', facecolor=[1, 0, 0, 0])
        poly2 = patches.Polygon(patch2, edgecolor=edgecolor, facecolor=facecolor)
        line = ax1.add_patch(poly1)
        line = ax1.add_patch(poly2)
    
    # plot location of blueseis in black
    locations = H_array2D(mult=mult)
    ax1.plot(0, 0, color='k', marker='o', zorder=19)
    
    ax1.set_aspect('equal')
    plt.xlim([-45 * mult, 55 * mult])
    plt.ylim([-45 * mult, 55 * mult])
    
    
    ax1.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    labelleft=False,
    labelcolor='white')
    ax.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    labelcolor='white')
#plt.show()
#fig.savefig('3D_modeshape_%s.png' %mode, dpi=300)